In [7]:
from pyhive import presto
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import geopandas as gpd
import pandas as pd
import osmnx as ox
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import io
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree
import folium
from zipfile import ZipFile

In [8]:
username = ''
password = ''
schema = 'arbi_opslogs'
conn =  presto.Connection(host='presto-secure.idata.shopeemobile.com', port=443, username= username ,password= password, schema = schema,
protocol='https')

In [9]:
df_dropoffpoints = pd.read_sql(f"""
select * from 
arbi_opslogs.batch_dropoff_points
limit 1825 
""", conn)

df_sellers = pd.read_sql(f"""
select * from 
arbi_opslogs.batch_sellers_points
""", conn)

/Users/milkomartorana/micromamba/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
df_dropoffpoints=df_dropoffpoints.rename(columns={'latitud':'lat','longitud':'lon'})
df_sellers=df_sellers.rename(columns={'latitud':'lat','longitud':'lon'})

In [11]:
convert_dict = {'shop_id': int,'lon': float,'lat': float}
df_sellers = df_sellers.astype(convert_dict)
print(df_sellers.dtypes)

convert_dict = {'nis':str,'lon': float,'lat': float}
df_dropoffpoints= df_dropoffpoints.astype(convert_dict)
print(df_dropoffpoints.dtypes)

shop_id                               int64
shop_name                            object
seller_id                            object
seller_name                          object
shipping_carrier                     object
seller_shipping_address              object
seller_shipping_address_state        object
seller_shipping_address_city         object
seller_shipping_address_district     object
seller_shipping_address_zipcode      object
seller_shipping_address_fully        object
min_cancel_day                       object
max_cancel_day                       object
orders                               object
total_gmv_usd                        object
days_with_cancels                    object
lat                                 float64
lon                                 float64
ingestion_timestamp                  object
dtype: object
nis                     object
codigo_de_planta        object
blindada                object
denominacion_           object
calle                   ob

In [12]:
m = folium.Map([-34.5742719,-58.6506512], tiles='CartoDb dark_matter')
locs_estaciones = zip(df_dropoffpoints.lat,df_dropoffpoints.lon)
locs_puntos = zip(df_sellers.lat, df_sellers.lon)
for location in locs_estaciones:
    folium.CircleMarker(location=location, 
        color='red',   radius=4).add_to(m)
for location in locs_puntos:
    folium.CircleMarker(location=location, 
        color='white', radius=2).add_to(m)
m

In [13]:
df_dropoffpoints = gpd.GeoDataFrame(df_dropoffpoints, geometry=gpd.points_from_xy(df_dropoffpoints.lon, df_dropoffpoints.lat))
df_sellers = gpd.GeoDataFrame(df_sellers, geometry=gpd.points_from_xy(df_sellers.lon, df_sellers.lat))

In [14]:
def dist(lat1, long1, lat2, long2):
    """
    Calculando distancia circular entre dos puntos (grados decimales)
    """
    # Para convertir de grados decimales a radianes
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # Formula de harvesine 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radio de la tierra en kilometros 6371
    km = 6371* c
    return km

In [15]:
def find_nearest(lat, long):
    distances = df_dropoffpoints.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    return df_dropoffpoints.loc[distances.idxmin(), 'nis']

In [16]:
df_sellers['nis'] = df_sellers.apply(
    lambda row: find_nearest(row['lat'], row['lon']), 
    axis=1)
# revisamos si nos devuelve el mas cercano
df_sellers.head()

,shop_id,shop_name,seller_id,seller_name,shipping_carrier,seller_shipping_address,seller_shipping_address_state,seller_shipping_address_city,seller_shipping_address_district,seller_shipping_address_zipcode,...,min_cancel_day,max_cancel_day,orders,total_gmv_usd,days_with_cancels,lat,lon,ingestion_timestamp,geometry,nis
0,638362569,HyT Electronics,638382154,u5kc0ons76,Andreani Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,2022-01-11 11:15:08,2022-01-11 11:31:17,4,3.667.465.466.109,4,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005
1,618572456,grupoferta,618592041,grupoferta,Correo Argentino Estándar a domicilio,"pasteur, 611, piso 8 86,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1028,...,2022-01-11 11:19:20,2022-01-11 11:19:20,1,719.313.921.052,1,-34.602026,-58.399657,1651171544,POINT (-58.39966 -34.60203),C0053
2,638362569,HyT Electronics,638382154,u5kc0ons76,Correo Argentino Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,2022-01-11 11:14:11,2022-01-11 11:31:21,13,4.889.055.384.164,13,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005
3,639312444,BA.SEX,639332029,ba.sex,Correo Argentino Estándar a domicilio,"Arenales, 3800, 4 b,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1425,...,2022-01-11 12:19:11,2022-01-11 12:19:11,1,36.520.664.064,1,-34.584687,-58.415024,1651171544,POINT (-58.41502 -34.58469),C0025
4,647081076,Mania-Electronic,647100661,mania_electronic,Correo Argentino Express a domicilio,"Gral Gervasio Artigas , 1177, , Local",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1416,...,2022-01-13 16:45:57,2022-01-13 16:45:57,1,398.094.715.982,1,-34.616927,-58.469559,1651171544,POINT (-58.46956 -34.61693),C5361


In [17]:
df_sellers = pd.merge(df_sellers,df_dropoffpoints[['nis','lat','lon','geometry']],on='nis', how='left')
# Renombro la columna iguales
df_sellers=df_sellers.rename(columns = {'lat_x':'m_lat','lon_x':'m_lon','lat_y':'h_lat','lon_y':'h_lon','geometry_x':'m_geometry','geometry_y':'h_geometry'})

In [18]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6371* c
    return km

# Output para ecuación.
df_sellers['distance'] = [haversine(df_sellers.m_lon[i],df_sellers.m_lat[i],df_sellers.h_lon[i],df_sellers.h_lat[i]) for i in range(len(df_sellers))]
df_sellers['distance'] = df_sellers['distance'].round(decimals=3)
# Printing the data table 
df_sellers.head()

,shop_id,shop_name,seller_id,seller_name,shipping_carrier,seller_shipping_address,seller_shipping_address_state,seller_shipping_address_city,seller_shipping_address_district,seller_shipping_address_zipcode,...,days_with_cancels,m_lat,m_lon,ingestion_timestamp,m_geometry,nis,h_lat,h_lon,h_geometry,distance
0,638362569,HyT Electronics,638382154,u5kc0ons76,Andreani Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,4,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005,-34.606197,-58.440973,POINT (-58.44097 -34.60620),0.916
1,618572456,grupoferta,618592041,grupoferta,Correo Argentino Estándar a domicilio,"pasteur, 611, piso 8 86,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1028,...,1,-34.602026,-58.399657,1651171544,POINT (-58.39966 -34.60203),C0053,-34.600977,-58.401793,POINT (-58.40179 -34.60098),0.228
2,638362569,HyT Electronics,638382154,u5kc0ons76,Correo Argentino Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,13,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005,-34.606197,-58.440973,POINT (-58.44097 -34.60620),0.916
3,639312444,BA.SEX,639332029,ba.sex,Correo Argentino Estándar a domicilio,"Arenales, 3800, 4 b,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1425,...,1,-34.584687,-58.415024,1651171544,POINT (-58.41502 -34.58469),C0025,-34.583906,-58.418196,POINT (-58.41820 -34.58391),0.303
4,647081076,Mania-Electronic,647100661,mania_electronic,Correo Argentino Express a domicilio,"Gral Gervasio Artigas , 1177, , Local",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1416,...,1,-34.616927,-58.469559,1651171544,POINT (-58.46956 -34.61693),C5361,-34.614199,-58.472267,POINT (-58.47227 -34.61420),0.392


In [71]:
df_sellers.to_csv("output.csv", sep='\t', encoding='utf-8')

In [19]:
df_sellers['ID'] = df_sellers.reset_index().index
df_sellers

,shop_id,shop_name,seller_id,seller_name,shipping_carrier,seller_shipping_address,seller_shipping_address_state,seller_shipping_address_city,seller_shipping_address_district,seller_shipping_address_zipcode,...,m_lat,m_lon,ingestion_timestamp,m_geometry,nis,h_lat,h_lon,h_geometry,distance,ID
0,638362569,HyT Electronics,638382154,u5kc0ons76,Andreani Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005,-34.606197,-58.440973,POINT (-58.44097 -34.60620),0.916,0
1,618572456,grupoferta,618592041,grupoferta,Correo Argentino Estándar a domicilio,"pasteur, 611, piso 8 86,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1028,...,-34.602026,-58.399657,1651171544,POINT (-58.39966 -34.60203),C0053,-34.600977,-58.401793,POINT (-58.40179 -34.60098),0.228,1
2,638362569,HyT Electronics,638382154,u5kc0ons76,Correo Argentino Estándar a domicilio,"Planes, 1078, Portón,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1405,...,-34.611804,-58.448304,1651171544,POINT (-58.44830 -34.61180),C0005,-34.606197,-58.440973,POINT (-58.44097 -34.60620),0.916,2
3,639312444,BA.SEX,639332029,ba.sex,Correo Argentino Estándar a domicilio,"Arenales, 3800, 4 b,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1425,...,-34.584687,-58.415024,1651171544,POINT (-58.41502 -34.58469),C0025,-34.583906,-58.418196,POINT (-58.41820 -34.58391),0.303,3
4,647081076,Mania-Electronic,647100661,mania_electronic,Correo Argentino Express a domicilio,"Gral Gervasio Artigas , 1177, , Local",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1416,...,-34.616927,-58.469559,1651171544,POINT (-58.46956 -34.61693),C5361,-34.614199,-58.472267,POINT (-58.47227 -34.61420),0.392,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,739822146,x6m5kp5rb6,739841731,gabialanis365,Correo en su casa,"24 de septiembre, 1052, 0,",SANTA FE,CORREA,,2506,...,-32.849527,-61.254021,1651171544,POINT (-61.25402 -32.84953),S0354,-32.850092,-61.250400,POINT (-61.25040 -32.85009),0.344,373
374,727023201,RUMISH,727042786,rumisho,Correo en su casa,"HUMAHUACA, 3433, ,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1191,...,-34.602946,-58.413435,1651171544,POINT (-58.41343 -34.60295),C0013,-34.603613,-58.416763,POINT (-58.41676 -34.60361),0.314,374
375,646160099,M&S Moda Sustentable,646179684,ms.moda.sustentable,Andreani en su casa,"Reconquista, 716, , Barrio Ledesma",JUJUY,LEDESMA,,4512,...,-23.832320,-64.789266,1651171544,POINT (-64.78927 -23.83232),Y0024,-23.814103,-64.794033,POINT (-64.79403 -23.81410),2.083,375
376,697093241,Tienda Tango Test,697112826,3tijw6x50a,Correo en su casa,"Cerrito, 1186, 2,",CAPITAL FEDERAL,CIUDAD AUTONOMA BUENOS AIRES,,1010,...,-34.594596,-58.383178,1651171544,POINT (-58.38318 -34.59460),C5043,-34.595784,-58.382881,POINT (-58.38288 -34.59578),0.135,376


In [63]:
# Create LineString Geometry
df_sellers['line'] = df_sellers.apply(lambda row: LineString([row['m_geometry'], row['h_geometry']]), axis=1)
# Create Line Geodataframe
line_gdf = df_sellers[['ID', 'h_geometry', 'line']].set_geometry('line')
# Set the Coordinate reference
line_gdf.crs = crs={'init':'epsg:4326'}


/Users/milkomartorana/micromamba/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/Users/milkomartorana/micromamba/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [66]:
cds = line_gdf.drop(columns=['h_geometry'])

In [70]:
m = folium.Map([-34.5742719,-58.6506512],zoom_start = 12,  
    tiles='CartoDb dark_matter')
locs_stations = zip(df_dropoffpoints.lat,df_dropoffpoints.lon)
locs_points = zip(df_sellers.m_lat, df_sellers.m_lon)
for location in locs_stations:
    folium.CircleMarker(location=location, 
              color='red',  radius=8).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
              color='white', radius=4).add_to(m)
folium.GeoJson(data=cds).add_to(m)
m.save('map2.html')

m